In [1]:
# Dependencies 
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [4]:
# Get url to scape
url= 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [5]:
# Collect the latest news title and paragraph
article = soup.find("div", class_='list_text')
title = article.find("div", class_="content_title").text
p = article.find("div", class_ ="article_teaser_body").text

# Print title and paragraph
print(title)
print('------------------------')
print(p)

NASA Readies Perseverance Mars Rover's Earthly Twin 
------------------------
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.


# JPL Mars Space Images - Featured Image

In [ ]:
# URL for featured image 
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(image_url)

# Go to full image
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)

# Go to more info
browser.click_link_by_partial_text('more info')


In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
image_soup = bs(html, 'html.parser')

# Scrape the URL
img_url = image_soup.find('figure', class_='lede').a['href']
featured_image_url = f'https://www.jpl.nasa.gov{img_url}'
print(featured_image_url)